<a href="https://colab.research.google.com/github/kataria342/nlp_project2/blob/main/UpdatedCode4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from textblob import TextBlob
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [5]:
# import nltk
# nltk.download('all')

In [7]:
symptomsDF1 = pd.read_csv('dataset.csv')
symptomsDF1 = symptomsDF1.replace(np.NaN, ' ')
cols = list(symptomsDF1.columns)
cols
for i in cols[1:]:
  symptomsDF1[i] = symptomsDF1[i].str.split("_")

symptomsDF1['combined_symptoms'] = symptomsDF1['Symptom_1'] + symptomsDF1['Symptom_2'] + symptomsDF1['Symptom_3'] + symptomsDF1['Symptom_4'] + symptomsDF1['Symptom_5'] + symptomsDF1['Symptom_6'] + symptomsDF1['Symptom_7'] + symptomsDF1['Symptom_8'] + symptomsDF1['Symptom_9'] + symptomsDF1['Symptom_10'] + symptomsDF1['Symptom_11'] + symptomsDF1['Symptom_12'] + symptomsDF1['Symptom_13'] + symptomsDF1['Symptom_14'] + symptomsDF1['Symptom_15'] + symptomsDF1['Symptom_16'] + symptomsDF1['Symptom_17']
symptomsDF1.drop(columns=cols[1:],inplace=True)
symptomsDF1['combined_symptoms'] = symptomsDF1['combined_symptoms'].apply(lambda x: ' '.join(x))


symptomsDF1.head(2)

,Disease,combined_symptoms
0,Fungal infection,itching skin rash nodal skin eruptions dischromic patches
1,Fungal infection,skin rash nodal skin eruptions dischromic patches


In [8]:
sample_date = pd.read_csv('Medical Data.csv')

symptomsDF2 = sample_date[['Diesase_Name','symptoms_elements', 'symptoms_paragraph']]
symptomsDF2['combined_symptoms'] = symptomsDF2['symptoms_elements'] + symptomsDF2['symptoms_paragraph']
symptomsDF2.drop(columns=['symptoms_elements','symptoms_paragraph'], inplace=True)

symptomsDF2.dropna(inplace=True)
symptomsDF2.columns = ['Disease','combined_symptoms']
symptomsDF2.head(1)


data = [symptomsDF2, symptomsDF1]
symptomsDF = pd.concat(data, ignore_index=True)
print(symptomsDF.size)
symptomsDF.drop_duplicates(inplace=True)
print(symptomsDF.size)
symptomsDF.head(2)
symptomsDF = symptomsDF.groupby('Disease')['combined_symptoms'].agg(' '.join).reset_index()
symptomsDF[symptomsDF['Disease'] == 'Fungal infection']


12734
2102


,Disease,combined_symptoms
296,Fungal infection,itching skin rash nodal skin eruptions dischromic patches skin rash nodal skin eruptions dischromic patches itching nodal skin eruptions dischromic patches itching skin rash dischromic patches itching skin rash nodal skin eruptions


In [9]:
def clean_duplicate(text):
  words = text.split()
  unique_words = set()
  result = []
  for word in words:
      if word not in unique_words:
          unique_words.add(word)
          result.append(word)

  return ' '.join(result)

symptomsDF['combined_symptoms'] = symptomsDF['combined_symptoms'].apply(clean_duplicate)


symptomsDF[symptomsDF['Disease'] == 'Fungal infection']

,Disease,combined_symptoms
296,Fungal infection,itching skin rash nodal eruptions dischromic patches


In [10]:
diseaseList = list(symptomsDF['Disease'].unique())
for i in diseaseList:
  print(i)

(vertigo) Paroymsal  Positional Vertigo
ACL injury
AIDS
ARDS
Abdominal aortic aneurysm
Absence seizure
Achalasia
Achilles tendon rupture
Acne
Acoustic neuroma
Acromegaly
Acute coronary syndrome
Acute kidney failure
Acute liver failure
Acute lymphocytic leukemia
Acute myelogenous leukemia
Acute sinusitis
Addison's disease
Adjustment disorders
Adrenal cancer
Adult Still disease
Adult attention-deficit/hyperactivity disorder (ADHD)
Agoraphobia
Albinism
Alcohol poisoning
Alcohol use disorder
Alcoholic hepatitis
Allergies
Allergy
Alpha-gal syndrome
Alzheimer's disease
Ambiguous genitalia
Ameloblastoma
Amenorrhea
Amnesia
Ampullary cancer
Amyloidosis
Amyotrophic lateral sclerosis (ALS)
Anal cancer
Anal fissure
Anal fistula
Anal itching
Anemia
Angelman syndrome
Angina
Anorexia nervosa
Anorgasmia in women
Anterior vaginal prolapse (cystocele)
Antisocial personality disorder
Anxiety disorders
Aortic valve disease
Aortic valve regurgitation
Aortic valve stenosis
Appendicitis
Arteriosclerosis / at

In [11]:
symptomsDF[(symptomsDF['Disease'] == 'Cholestasis of pregnancy') | (symptomsDF['Disease'] == 'Chronic cholestasis') | (symptomsDF['Disease'] == 'Cholangiocarcinoma (bile duct cancer)')]

,Disease,combined_symptoms
156,Cholangiocarcinoma (bile duct cancer),"Yellowing of your skin and the whites eyes (jaundice)Intensely itchy skinWhite-colored stoolsFatigueAbdominal pain on right side, just below ribsLosing weight without tryingFeverNight sweatsDark urineSigns symptoms cholangiocarcinoma include:"
158,Cholestasis of pregnancy,"Yellowing of the skin and whites eyes, called jaundiceNauseaLoss appetiteOily, foul-smelling stoolsIntense itching is main symptom cholestasis pregnancy. But there no rash. Typically, you feel itchy on palms your hands or soles feet, but may everywhere. The often worse at night bother so much that can't sleep."
159,Chronic cholestasis,itching vomiting yellowish skin nausea loss of appetite abdominal pain yellowing eyes


In [12]:
def cleandata1(text):
    tag_mapping = {
    'NN': 'n',
    'NNS': 'n',
    'NNP': 'n',
    'NNPS': 'n',
    'VB': 'v',
    'VBD': 'v',
    'VBG': 'v',
    'VBN': 'v',
    'VBP': 'v',
    'VBZ': 'v',
    'JJ': 'a',
    'JJR': 'a',
    'JJS': 'a',
    'RB': 'r',
    'RBR': 'r',
    'RBS': 'r',
    'WRB':'r'
    }
    text = "".join([word.lower() for word in re.sub(r"[^a-zA-Z]", " ", text)])
    tokens = word_tokenize(text)
    text = [word for word in tokens if len(word)>2 or word not in stop_words]
    text = " ".join([lemmatizer.lemmatize(word, pos='n' if tag_mapping.get(TextBlob(word).tags[0][1]) == None else tag_mapping.get(TextBlob(word).tags[0][1])) for word in text])
    return text
symptomsDF['combined_symptoms'] = symptomsDF['combined_symptoms'].apply(lambda x: cleandata1(x))

symptomsDF.head(2)

,Disease,combined_symptoms
0,(vertigo) Paroymsal Positional Vertigo,vomit headache nausea spin movement loss balance unsteadiness
1,ACL injury,loud pop pop sensation the kneesevere pain and inability continue activityrapid swellingloss range motiona feel instability give way with weight bearingsigns symptom acl injury usually include


In [13]:
diseaseDescriptionDF1 = pd.read_csv('symptom_Description.csv')
diseaseDescriptionDF2 = sample_date[['Diesase_Name','overview']]
diseaseDescriptionDF2.columns = ['Disease','Description']
data = [diseaseDescriptionDF2, diseaseDescriptionDF1]
diseaseDescriptionDF = pd.concat(data, ignore_index=True)
diseaseDescriptionDF.drop_duplicates(inplace=True)
diseaseDescriptionDF.set_index('Disease', inplace = True)
diseaseDescriptionDF.head(2)

,Description
Disease,
Atrial fibrillation,"In a typical heart, a tiny group of cells at the sinus node sends out an electrical signal. The signal travels through the heart's upper chambers to the atrioventricular (AV) node. The signal then passes into the lower heart chambers, causing them to squeeze and pump out blood. In atrial fibrillation, electrical signals fire from many locations in the upper chambers, causing them to beat chaotically. Since the AV node doesn't prevent all of these chaotic signals from entering the lower chambers, the heart beats faster and irregularly."
Abdominal aortic aneurysm,"An abdominal aortic aneurysm occurs when a lower part of the body's main artery, called the aorta, becomes weakened and bulges."


In [149]:
# def CorrectSpellings(text):
#     tokens = [str(TextBlob(word).correct()) for word in word_tokenize(text)]
#     text = " ".join(tokens)
#     return text

# diseaseDescriptionDF['Disease'] = diseaseDescriptionDF['Disease'].apply(lambda X: CorrectSpellings(X))
# symptomsDF['Disease'] = symptomsDF['Disease'].apply(lambda X: CorrectSpellings(X))

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
embed = model(symptomsDF['combined_symptoms'])


2024-01-11 07:16:15.939810: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 07:16:16.001773: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 07:16:16.001821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 07:16:16.004006: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-11 07:16:16.013818: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-11 07:16:16.015090: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [15]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1)

neigh.fit(embed)

NearestNeighbors(n_neighbors=1)

In [16]:
symptoms = input('enter your symptoms: \n')
print('Symptoms Entered: ', symptoms)
symptoms = cleandata1(symptoms)
print(symptoms)
emd = model([symptoms])
nei = neigh.kneighbors(emd,return_distance=False)[0][0]
disease_name = symptomsDF['Disease'].iloc[nei]
description = diseaseDescriptionDF['Description'].loc[disease_name]
print('**********************************')
print('Predicted Disease : ',disease_name.title())
print('Disease Definition:',description)
print('**********************************')

Symptoms Entered:  i have hot forhead and also sweating
have hot forhead and also sweat
**********************************
Predicted Disease :  Hyperhidrosis
Disease Definition: Hyperhidrosis (hi-pur-hi-DROE-sis) is excessive sweating that's not always related to heat or exercise. You may sweat so much that it soaks through your clothes or drips off your hands. Heavy sweating can disrupt your day and cause social anxiety and embarrassment.
**********************************


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(embed, emd)

In [19]:
max = cosine_similarity_matrix[0]
index = 0
for i in range(1,len(cosine_similarity_matrix)):
    if cosine_similarity_matrix[i] > max:
        max = cosine_similarity_matrix[i]
        index = i
print(f'Index of the maximum value is : {index}')
symptomsDF['Disease'].iloc[index]

Index of the maximum value is : 367


'Hyperhidrosis'

In [20]:
testes={
    "Fungal infection" : "My skin has been itchy and flaky with a reddish rash that doesn't seem to heal.",
"Allergy" : "Whenever I'm around pollen, my eyes water, my nose runs, and I can't stop sneezing.",
"GERD" : "I often experience a burning sensation in my chest, especially after eating, and sometimes food seems to come back up my throat.",
"Chronic cholestasis" : "I've been feeling persistent itching, and my eyes and skin appear more yellow than usual and I lost my appetite",
"Drug Reaction" : "After taking that medication, I developed a rash, swelling, and felt dizzy.",
"Peptic ulcer disease" : "There's a gnawing or burning pain in my stomach, especially on an empty stomach or at night.",
"AIDS (Acquired Immune Deficiency Syndrome)" : "I've lost weight without trying, feel constantly fatigued, and keep getting infections that I never used to have.",
"Diabetes" : "I've been extremely thirsty, urinating often, and feeling more tired than usual.",
"Gastroenteritis" : "I can't keep anything down, have diarrhea, and feel abdominal cramps.",
"Bronchial Asthma" : "I often wheeze and have shortness of breath, especially during the night or after exercise.",
"Hypertension (High blood pressure)" : "The doctor said my blood pressure readings are consistently high, even though I don't feel any specific symptoms.",
"Migraine" : "There's a throbbing pain on one side of my head, and I become sensitive to light and sound.",
"Cervical spondylosis" : "My neck feels stiff and painful, and sometimes I feel tingling or numbness in my hands.",
"Paralysis (brain hemorrhage)" : "Half of my body suddenly became weak, and I couldn't move my right arm or leg.",
"Jaundice" : "My skin and eyes have taken on a yellow tint, and my urine is much darker than usual.",
"Malaria" : "I've had recurring bouts of high fever, chills, and sweats for the past few days.",
"Chicken pox" : "I've broken out in itchy blisters all over my body, and I had a fever before the rash appeared.",
"Dengue" : "I'm feeling sudden high fever, severe headaches, and pain behind my eyes, along with joint and muscle pain.",
"Typhoid" : "I've been suffering from a persistent fever, stomach pain, headache, and I noticed a rash on my abdomen.",
"Hepatitis A" : "I've lost my appetite, feel nauseous, and my liver area hurts; plus, I've noticed a yellowing of my eyes.",
"Hepatitis B" : "I've been feeling very tired, my urine is dark, and there's a yellowish tinge to my eyes and skin.",
"Hepatitis C" : "I'm feeling a general sense of fatigue, my stomach is upset, and the doctor said my liver enzymes are elevated.",
"Hepatitis D" : "I've had jaundice, fatigue, and joint pain recently, even though I already had hepatitis B.",
"Hepatitis E" : "I've noticed jaundice, feel fatigued, and have a reduced appetite along with mild fever.",
"Alcoholic hepatitis" : "After years of heavy drinking, I've been feeling nauseous, have abdominal pain, and my eyes and skin are turning yellow.",
"Tuberculosis" : "I've had a persistent cough for weeks, sometimes coughing up blood, accompanied by weight loss, night sweats, and fatigue.",
"Common Cold" : "I've got a runny nose, sneezing, a mild sore throat, and a cough that just started.",
"Pneumonia" : "I've been feeling short of breath, with a high fever and a cough that brings up thick, colored phlegm.",
"Dimorphic hemorrhoids (piles)" : "I've noticed painful swollen veins in my rectal area, and sometimes there's blood when I wipe.",
"Heart attack" : "I suddenly felt a crushing pain in my chest that radiated to my arm and jaw, accompanied by shortness of breath.",
"Varicose veins" : "The veins in my legs have become bulgy, bluish, and often cause a dull ache.",
"Hypothyroidism" : "I've been feeling constantly tired, gaining weight, and my skin has turned dry and cold.",
"Hyperthyroidism" : "I've lost weight without trying, my heart rate has increased, and I feel jittery and hot all the time.",
"Hypoglycemia" : "I suddenly felt shaky, sweaty, and had a pounding heartbeat, and needed to eat something sweet immediately.",
"Osteoarthritis" : "The joints in my hands and knees have become painful and stiff, especially when I wake up.",
"Arthritis" : "My joints are swollen, red, and warm to the touch, and they ache constantly.",
"(Vertigo) Paroxysmal Positional Vertigo" : "Whenever I change the position of my head, I feel like the room is spinning around me.",
"Acne" : "My face, back, and chest have red, pus-filled pimples and blackheads.",
"Urinary tract infection" : "I've had a burning sensation when I urinate, and I feel the urge to go more often than usual.",
"Psoriasis" : "There are red, itchy patches covered with silvery scales on my elbows and knees.",
"Impetigo" : "I've developed honey-colored crusted sores on my face, especially around my nose and mouth."

}



emd = model([symptoms])
nei = neigh.kneighbors(emd,return_distance=False)[0][0]
disease_name = symptomsDF['Disease'].iloc[nei]
count=0
for k,v_text in testes.items():
    symptoms = cleandata1(v_text)
    emd = model([symptoms])
    nei = neigh.kneighbors(emd,return_distance=False)[0][0]
    _ct = symptomsDF['Disease'].iloc[nei]
    r= "✅" if _ct == k else "❌"
    if("✅"==r):
        count+=1
    print(r , end=" ")
    print(f"Predicted: {_ct}, Original: {k},  \n\tSentence: {v_text}")
print(f"Total {count}/{len(testes)}")

❌ Predicted: Drug Reaction, Original: Fungal infection,  
	Sentence: My skin has been itchy and flaky with a reddish rash that doesn't seem to heal.
✅ Predicted: Allergy, Original: Allergy,  
	Sentence: Whenever I'm around pollen, my eyes water, my nose runs, and I can't stop sneezing.
❌ Predicted: Gastroesophageal reflux disease (GERD), Original: GERD,  
	Sentence: I often experience a burning sensation in my chest, especially after eating, and sometimes food seems to come back up my throat.
❌ Predicted: Cholangiocarcinoma (bile duct cancer), Original: Chronic cholestasis,  
	Sentence: I've been feeling persistent itching, and my eyes and skin appear more yellow than usual and I lost my appetite
❌ Predicted: Balance problems, Original: Drug Reaction,  
	Sentence: After taking that medication, I developed a rash, swelling, and felt dizzy.
❌ Predicted: Peptic ulcer, Original: Peptic ulcer disease,  
	Sentence: There's a gnawing or burning pain in my stomach, especially on an empty stoma